# 案例: 参数自动校准 (独立脚本版)

本案例再次演示了如何使用`ParameterIdentificationAgent`来自动校准模型参数。与之前的版本相比，这个案例更进一步，它会计算并打印出模型在校准前后的均方根误差（RMSE），以量化性能的提升，并使用断言（assert）来验证校准的成功。

### 1. 导入库并加载配置

In [ ]:
import json
import numpy as np

from swp.simulation_identification.physical_objects.rainfall_runoff import RainfallRunoff
from swp.simulation_identification.identification.identification_agent import ParameterIdentificationAgent
from swp.central_coordination.collaboration.message_bus import MessageBus
from swp.data_processing.evaluator import calculate_rmse

CONFIG_PATH = 'examples/parameter_identification_standalone.json'
with open(CONFIG_PATH, 'r') as f:
    config = json.load(f)

print("配置加载成功！")

### 2. 定义并运行仿真

In [ ]:
def run_standalone_id_example(cfg):
    # 1. 定义真实参数并生成数据
    true_params = cfg['ground_truth']
    data_cfg = cfg['data_generation']
    np.random.seed(data_cfg['seed'])
    rainfall_series = np.random.rand(data_cfg['num_points']) * data_cfg['max_rainfall_intensity']
    observed_runoff_series = true_params['runoff_coefficient'] * rainfall_series * true_params['catchment_area']

    # 2. 创建带有错误初始参数的模型
    initial_wrong_coeff = cfg['model_initial_guess']['runoff_coefficient']
    model_params = {'catchment_area': true_params['catchment_area'], 'runoff_coefficient': initial_wrong_coeff}
    runoff_model = RainfallRunoff(name="catchment_to_calibrate", parameters=model_params)
    print(f"模型初始径流系数: {runoff_model.get_parameters()['runoff_coefficient']:.2f}")

    # 3. 评估初始模型性能
    initial_simulated_runoff = initial_wrong_coeff * rainfall_series * true_params['catchment_area']
    initial_rmse = calculate_rmse(initial_simulated_runoff, observed_runoff_series)
    print(f"初始均方根误差 (RMSE): {initial_rmse:.4f}")

    # 4. 设置辨识智能体
    bus = MessageBus()
    agent_cfg = cfg['agent_config']
    id_agent = ParameterIdentificationAgent(
        agent_id="id_agent_1",
        target_model=runoff_model,
        message_bus=bus,
        config=agent_cfg
    )

    # 5. 模拟数据流并运行智能体
    print("\n--- 模拟数据流并运行辨识过程 ---")
    data_map = agent_cfg['data_map']
    for i in range(len(rainfall_series)):
        bus.publish(data_map['rainfall'], {'value': rainfall_series[i]})
        bus.publish(data_map['observed_runoff'], {'value': observed_runoff_series[i]})
    id_agent.run(current_time=1)

    # 6. 评估最终模型性能
    final_coeff = runoff_model.get_parameters()['runoff_coefficient']
    print(f"\n最终辨识出的径流系数: {final_coeff:.2f}")
    final_simulated_runoff = final_coeff * rainfall_series * true_params['catchment_area']
    final_rmse = calculate_rmse(final_simulated_runoff, observed_runoff_series)
    print(f"最终均方根误差 (RMSE): {final_rmse:.4f}")
    
    # 7. 断言验证
    assert final_rmse < initial_rmse
    assert abs(final_coeff - true_params['runoff_coefficient']) < 1e-3
    print("\n示例成功结束。智能体正确地辨识出了参数，且误差减小。")

run_standalone_id_example(config)